In [25]:
import os
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [5]:
list_filename="gspy-db-20180813_O1_filtered_t1126400691-1205493119_snr7.5_tr_gspy.csv"

In [ ]:
data_dir = os.path.join(os.path.dirname(os.getcwd()),"data")

In [7]:
gl_df = pd.read_csv(os.path.join(data_dir,list_filename))
gl_df.describe()

,Index,GPStime,peakFreq,snr,amplitude,centralFreq,duration,bandwidth,chisq,chisqDof,confidence
count,81701.000000,8.170100e+04,81701.000000,81701.000000,8.170100e+04,81701.000000,81701.000000,81701.000000,81701.0,81701.0,81701.000000
mean,40850.000000,1.164020e+09,600.612925,25.861831,3.486329e-20,1936.617241,1.271748,3128.907362,0.0,0.0,0.999152
std,23585.191509,1.646547e+07,575.187632,169.987708,2.669169e-19,1360.470233,2.141393,2816.322956,0.0,0.0,0.001235
min,0.000000,1.126403e+09,10.059000,7.500000,2.260000e-23,8.319000,0.001000,0.495630,0.0,0.0,0.996000
25%,20425.000000,1.163774e+09,38.897000,8.212000,1.460000e-22,1015.524000,0.164000,281.646606,0.0,0.0,0.999000
50%,40850.000000,1.165438e+09,262.065000,9.363000,1.980000e-22,1526.092000,0.500000,2451.317383,0.0,0.0,1.000000
75%,61275.000000,1.168694e+09,1085.830000,12.598000,3.800000e-22,3234.508000,1.625000,5722.214400,0.0,0.0,1.000000
max,81700.000000,1.205493e+09,2046.281000,11499.370000,2.780000e-17,4727.692000,64.500000,7946.482910,0.0,0.0,1.000000


In [8]:
gl_df.shape

(81701, 15)

In [9]:
X = gl_df.get(['GPStime','peakFreq','snr','amplitude','centralFreq','duration','bandwidth','chisq','chisqDof','confidence'])
Y = gl_df.get('label')

print('X : ')
print(X.count())
print()
print('Y: ')
print(Y.count())

Y_num = pd.factorize(Y)
Y_num_labels = Y_num[0]
Y_count_labels = Y_num[1].size

print()
print('Distinct classes :', Y_count_labels)
print()
print('Y_num_labels : ', Y_num_labels.size)

X : 
GPStime        81701
peakFreq       81701
snr            81701
amplitude      81701
centralFreq    81701
duration       81701
bandwidth      81701
chisq          81701
chisqDof       81701
confidence     81701
dtype: int64

Y: 
81701

Distinct classes : 22

Y_num_labels :  81701


In [10]:
seed = 7
test_size = 0.33
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_num_labels, test_size=test_size, random_state=seed)

In [11]:
print('X_train : ')
print(X_train.count())
print()
print('Y_train: ')
print(Y_train.size)
print()
print('X_test: ')
print(X_test.count())
print()
print('Y_test: ')
print(Y_test.size)

X_train : 
GPStime        54739
peakFreq       54739
snr            54739
amplitude      54739
centralFreq    54739
duration       54739
bandwidth      54739
chisq          54739
chisqDof       54739
confidence     54739
dtype: int64

Y_train: 
54739

X_test: 
GPStime        26962
peakFreq       26962
snr            26962
amplitude      26962
centralFreq    26962
duration       26962
bandwidth      26962
chisq          26962
chisqDof       26962
confidence     26962
dtype: int64

Y_test: 
26962


In [22]:
rf = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
rf.fit(X_train, Y_train)
rf_preds = rf.predict(X_test)

In [23]:
error_rate_rf = np.sum(rf_preds != Y_test) / Y_test.size
error_rate_rf

0.19008233810548178

In [24]:
rf.feature_importances_

array([0.12820822, 0.39050001, 0.07403522, 0.        , 0.11118318,
       0.16849402, 0.07311858, 0.        , 0.        , 0.05446077])

In [27]:
print(classification_report(Y_test, rf_preds))

              precision    recall  f1-score   support

           0       0.50      0.00      0.01       231
           1       0.81      0.31      0.45      1265
           2       0.72      0.97      0.83      4285
           3       1.00      0.01      0.01       555
           4       0.73      0.77      0.75      1277
           5       0.88      0.59      0.71      1277
           6       0.87      0.70      0.77       945
           7       0.90      1.00      0.95     10032
           8       0.95      0.11      0.19       825
           9       0.00      0.00      0.00        56
          10       0.00      0.00      0.00       130
          11       0.00      0.00      0.00        33
          12       0.93      0.76      0.83       878
          13       0.70      0.90      0.78      2730
          14       0.71      0.82      0.76      1909
          15       0.00      0.00      0.00       235
          16       1.00      0.26      0.41       276
          17       0.00    

/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
